# Python Project  

Kwangwoon University 전자통신공학과 3학년 하계 미니 프로젝트  
2014707073 김수환

## Global Variable and Const  
프로그램이 돌아가는데 필요한 전역변수 및 상수 정의 모음

In [1]:
import copy
DUP_SIZE   =  3
KEYWORD    =  ['+','-','/','*','//','%','=','==',':','if','for','elif','else','continue','"','\'','\'''','"""',
               'and','or','&','(',')',':','[',']','\\','\r','\f','\a','\b','\000', '\t','%d','%f','%c','%s','<','>','<=','>=',
               '%o','%x','%%','.',',','!=','not','pass',''
              ,'format','count','find','index','join','upper','lower','rstrip','strip','replace','split','len','str','del'
              ,'append','sort','reverse','insert','remove','pop','extend','keys','values','items','clear','get','in','set'
               ,'|','union','difference','add','update','True','False','type'
               ,'while','if','elif','else','print','input','bool','id','is','copy','int','break','for','range','def'
              ,'return','lambda','map','end','open','close','\'r\'','\'w\'','\'a\'','readline','read','write','with','as'
              ,'global','class','__init__','import','from','__name__=="__main__"','__name__ == "__main__"'
              ,'try','except','finally','abs','all','any','chr','dir','divmod','enumerate','eval','filter','hex','isinstance'
              ,'list','max','min','oct','ord','pow','round','sorted','sum','tuple','zip','sys','pickle','os','shutil'
               ,'glob','tempfile','time','calendar','random','webbrowser', 'STRING', 'VAR', 'CONST', 'TAB', ';', ':']

## Preprocess
텍스트를 전처리해주는 함수  
알파벳 및 숫자를 제외한 특수문자들은 양 옆에 공백을 추가로 끼워넣는다

In [2]:
def Preprocess(text):
    except_list    =   ['_', '\t', '\n']             #  예외처리를 위한 리스트
    preprocessed   =   ""                            #  text를 전처리해서 저장할 문자열
    text           =   text.replace('    ',' TAB ').replace('\t',' TAB ')   #  space 4개를 TAB으로 치환한다
                                                     #  뒤에 함수에서 이를 진짜 '\t'으로 치환한다
                                                     #  여기서 바로 치환해버리면 뒤의 split()에서 없어져버린다
    # 텍스트를 char 단위로 읽는다
    for char in text:
        # 읽은 char이 알파벳도 아니고 숫자도 아니고 except_list에도 존재하지 않는다면
        if char.isalpha() == False and char.isdigit() == False and char not in except_list:
            # 이따 split()으로 구분하기 위해 양 옆으로 " "을 추가한다
            preprocessed += (" " + char + " ")
        else:
            # 위의 경우가 아니라면 연속된 단어라고 판단하고 " "없이 붙여넣는다
            preprocessed += char
    # 전처리된 텍스트를 반환한다        
    return preprocessed

## Set Number

코드를 라인을 표시하여 보여주는 함수 (디버그용)

In [3]:
def set_number(line):
    for i, item in enumerate(line):
        print(i+1, ": ", item)

# Text-to-Line  

전처리 된 텍스트를 라인별로 나눠주는 함수

In [4]:
def Text_to_line(preprocessed):
    line    =   []       #  라인별로 저장할 리스트
    buf     =   ""       #  line에 넣을 문자열. 특정 케이스는 골라내기 위해 임시 저장
    quote   =   False   #   ' 이나 " 에 감싸진 문자열을 모두 STRING으로 바꿔주기 위한 '플래그'
    mark    =   False   #   주석처리는 검사하지 않도록 하기 위한 '플래그'
    
    # 전처리된 텍스트를 char 단위로 읽는다
    for i, char in enumerate(preprocessed):
        # 개행 문자가 나오면 1 라인이라 생각하고 line에 집어넣는다
        if char == '\n' and quote == False:
            mark = False
            buf += char
            # 만약 1라인이 '\n'이 전부라면 line에 추가하면 뒤에 
            # 중복 검사를 할 때 오류가 발생한다.
            # 그러므로 split()로 나눠봐서 길이가 0이라면 ('\n'이 전부라면) 넘어가고
            # 길이가 0이 아니라면 적절한 코드라고 판단하고 line에 집어넣는다
            if len(buf.split()) != 0:
                line.append(buf)
            # 1라인이 끝났으므로 buf를 초기화 해준다
            buf = ""
            continue
        # 주석처리라면 넘어간다
        if mark == True:
            continue
        # 문자열을 STRING으로 치환하기 위한 플래그 작업
        if char == '#':
            mark = True
            continue
        if (char == '\'' or char == '\"'):
            # 플래그가 on인 상태에서 또 들어온다면 buf에 'STRING' 을 추가하고 플래그를 off 한다
            if quote == True:
                buf += 'str'
                quote = False
            # 플래그가 off인 상황에서 들어온다면 플래그를 on 해준다
            else:
                quote = True
            continue
            
        # 문자열이 아닌 공백은 buf에 추가한다    
        if char == ' ' and quote == False:
            buf += char
            continue

        # 현재 문자열이라고 판단 된 것이므로 넘어간다
        if quote == True:
            continue
        # 보통의 경우에는 buf에 char를 계속 추가한다
        buf += char
    # 코드 마지막이 '\n'이 아닐수도 있으므로 현재까지 저장된
    # buf를 마지막으로 추가해준다
    line.append(buf)
    # line을 반환한다
    return line

## Line to Key    
line을 입력으로 하여 변수나 상수를 판단하여 'VAR', 'CONST'로 치환해주는 함수   
변수나 상수를 치환하지 않고 검사시, 완전히 똑같은 코드가 아니라면 중복검사에
걸리지 않으므로, 치환이 필요함

In [5]:
def Line_to_key(line):
    changed  =  []   #  line을 적절히 변형하여 저장할 리스트
    original =  []   #  line을 원래 그대로 저장할 리스트
    
    # line에 저장된 문자열들을 ' '기준으로 나눈다
    for str_ in line:
        changed.append(str_.split())
        original.append(str_.split())
    
    # 쪼개진 line을 in 연산자를 이용해 하나하나 가져온다
    # 이때 가져온 단위는 list이다.
    for i, list_ in enumerate(changed):
        n = 1
        # list에서 한 번 더 in 연산자를 이용하여
        # 문자열 단위로 가져온다
        for j, str_ in enumerate(list_):
            # KEYWORD에 포함되는지를 표시할 플래그
            isKey = False
            # 만약 가져온 문자열이 KEYWORD에 포함된다면
            # 플래그 on
            if str_ in KEYWORD:
                isKey = True
            # 가져온 문자열이 KEYWORD가 아니라면
            if isKey == False:
                # 숫자인지를 검사해서 숫자라고 판단되면
                if changed[i][j].isdigit() == True:
                    # CONST로 치환한다
                    changed[i][j] = 'CONST'
                # 현재 문자열 다음 문자열이 '('이라면 
                # 함수라는 뜻이므로 그냥 내버려둔다
                elif j+1 < len(changed[i]) and changed[i][j+1] == '(':
                    continue
                # 키워드 x, 숫자 x 이므로 변수라고 판단하고
                # 'VAR'로 치환하는데, 적어도 같은 문장에서는
                # 변수를 구분해야 하므로, 뒤에 숫자를 붙여준다
                else:
                    changed[i][j] = 'var'
                    n += 1
            # 앞에서 치환해놓은 TAB을 진짜 '\t'으로 치환한다
            if str_ == 'TAB':
                changed[i][j] = '\t'
                original[i][j] = '\t'
                
    # changed랑 original을 반환
    return changed, original

## Check Duplicate
중복되는 코드들을 찾는 함수

In [6]:
def Check_duplicate(changed, original):
    duplicated        =    []   #  중복이라고 판단한 코드를 넣는 리스트
    original_dup      =    []   #  중복이라고 판단한 코드의 original 리스트
    doubtful          =    []   #  중복이 의심되는 코드를 넣는 리스트
    original_doubt    =    []   #  중복이 의심되는 코드의 original 리스트
    cursor            =    -1   #  현재 검사하는 코드 라인을 기록하는 리스트

    # 공백 기준으로 쪼개높은 splited의 요소들을
    # in 연산자로 changed_list1d로 가져온다
    for i, changed_list1d in enumerate(changed):
        # 이미 검사가 끝난 코드는 점프를 해주기 위함
        if cursor >= i:
            continue
        cursor += 1
        # changed_list1d와 비교할 list를 가져온다
        # changed_list1d의 다음 라인부터 마지막 라인까지 반복한다
        for j in range(i+1, len(changed)):
            # 이미 검사가 끝난 코드는 점프를 해주기 위함
            if cursor >= j:
                continue
            # changed_list1d와 똑같다고 판단된
            # 코드를 만나게 되면
            if changed_list1d == changed[j]:
                # 일단 doubtful 리스트에 추가를 한다
                doubtful.append(changed_list1d)
                original_doubt.append(original[i])
                # 그리고 origin, compare 모두 한 라인씩 증가하면서
                # 서로 비교해서 다른 코드가 나올 때까지 반복한다
                compare_index = j + 1
                original_index = i + 1
                while ( compare_index < len(changed) ) and changed[original_index] == changed[compare_index]:
                    cursor += 1
                    # 같은 코드가 나오는대로, doubtful 리스트에 추가해놓는다
                    doubtful.append(changed[original_index])
                    original_doubt.append(original[original_index])
                    compare_index += 1
                    original_index += 1

            # doubtful이 DUP_SIZE (중복처리할 기준) 보다 크거나 같고
            # 현재 duplicated에 이미 들어가있지 않는 코드라면
            # duplicated 리스트에 추가한다
            if len(doubtful) >= DUP_SIZE and doubtful not in duplicated:
                duplicated.append(doubtful)
                original_dup.append(original_doubt)

            doubtful = []          #  리스트 초기화
            original_doubt = []    #  리스트 초기화
    return duplicated, original_dup

## Print Duplicated
중복되었던 코드를 출력해주는 함수

In [7]:
def Print_duplicated(duplicated):
    print("duplicated:\n")
    for i,list2d in enumerate(duplicated):
        print(i+1, ":")
        for list1d in list2d:
            print('\t', end = "")
            for _str in list1d:
                print(_str, end = " ")
            print()

## _Create func Name  
함수 이름 및 파라미터를 형식에 맞게 만들어주는 함수  
(Create_func()에서 내부적으로 호출하는 함수)

In [8]:
def _Create_func_name(list2d_, n):
    func_  =  "func" + str(n) + "("
    start  =  True
    for list1d_ in list2d_:
        for _str in list1d_:
            if _str in change_var and _str not in func_:
                if start == True:
                    func_ += _str
                    start = False
                else:
                    func_ += (', ' + _str)
    func_ += ")"    
    return func_

## _Insert Space to Match Grammar
파이썬 문법에 맞게 적절한 공백 삽입을 해주는 함수  
(Create_func()에서 내부적으로 후출하는 함수)

In [9]:
def _Insert_Space_to_match_grammar(list2d_, append):
    no_space     =   ['.', '(', '[' ]                    #  앞, 뒤 공백이 필요없는 리스트
    rear_space   =   [')', ']', '=', '<', '>']           #  뒤에 공백이 필요한 리스트
    char2_case   =   ['<', '=', '>', '!', '+', '-']      #  '<=', '==' 같이 2 연산자가 연속 가능한 리스트
    new_func     =   ""
    for i, list1d_ in enumerate(list2d_):
        for index, str_ in enumerate(list1d_):
            if str_ in no_space:
                new_func += str_
            elif index + 1 < len(list1d_) and str_ == '=' and list1d_[index+1] == '[':
                new_func += (str_ + ' ')
            elif index + 1 < len(list1d_) and str_ in rear_space and list1d_[index+1] in no_space:
                new_func += str_
            elif index + 1 < len(list1d_) and str_ in char2_case and list1d_[index+1] == '=':
                new_func += str_
            elif str_ in rear_space:
                new_func += (str_ + ' ')
            elif index + 1   < len(list1d_) and list1d_[index+1] in char2_case:
                new_func += (str_ + ' ')
            elif index + 1 < len(list1d_) and list1d_[index+1] in no_space:
                new_func += str_
            elif index + 1 < len(list1d_) and list1d_[index+1] in rear_space:
                new_func += str_
            else:
                new_func += (str_ + ' ')
        new_func += '\n'
        if (i != len(list2d_) -1) :
            new_func += append
    return new_func

## Create Func
중복되었던 코드를 바탕으로 새로운 함수를 생성해주는 함수

In [10]:
def Create_func(func_ready, change_var):
    n                =   1       #  func에 번호를 매길 변수
    func_name_list   =   []      #  함수이름들을 저장할 리스트
    func_list        =   []      #  new_func로 생성한 함수들을 저장할 리스트
    
    for list2d_ in func_ready:
        new_func = "def "
        func_ = _Create_func_name(list2d_, n)
        func_name_list.append(func_)
        new_func += (func_ + " : \n\t")
        n += 1
        insert_space = _Insert_Space_to_match_grammar(list2d_, '\t')
        new_func += insert_space
        func_list.append(new_func)
    return func_list, func_name_list

## Delete Duplicate  
중복이라고 판단한 코드중에 겹치는 코드는 삭제처리 하는 함수

In [11]:
def del_duplicate(duplicate, original_dup):
    for i, func in enumerate(duplicated):
        if i < len(duplicated) - 1:
            for j in range(i + 1, len(duplicated)):
                isExist = True
                compare = duplicated[j]
                if len(func) > len(compare):
                    for list_ in func:
                        if list_ not in compare:
                            isExist = False
                    if isExist == True:
                        print('func    : ', func)
                        print('compare : ', compare)
                        del duplicated[j]
                        del original_dup[j]
                elif len(func) < len(compare):
                    for list_ in func:
                        if list_ not in compare:
                            isExist = False
                    if isExist == True:
                        print('func    : ', func)
                        print('compare : ', compare)
                        del duplicated[i]
                        del original_dup[i]
    return duplicate, original_dup

## _Find index  
list에서 word가 있는 인덱스를 반환하는 함수  
(check_same_var() 에서 내부적으로만 호출)

In [12]:
def _find_index(_list, word):
    for index, item in enumerate(_list):
        if item == word:
            return index
    return -1

## Check Same variable  
모든 변수들이 'VAR'로 변환되었던 변수들을 기존 변수로 바꾸는 함수 

In [13]:
def check_same_var(original_dup):
    origin_var  =  []
    func_var  =  []
    cnt         =   1
    text        =  ""
    
    for i, func in enumerate(original_dup):
        for j, line in enumerate(func):
            for k, word in enumerate(line):
                if word not in KEYWORD and not word.isdigit() and word not in origin_var:
                    next_index = k + 1
                    if next_index < len(line) and original_dup[i][j][next_index] != '(':
                        origin_var.append(word)
                        func_var.append('var' + str(cnt))
                        cnt += 1
                    elif next_index == len(line):
                        origin_var.append(word)
                        func_var.append('var' + str(cnt))
                        cnt += 1
    
    for i, func in enumerate(original_dup):
        for j, line in enumerate(func):
            for k, word in enumerate(line):
                if word not in KEYWORD and word in origin_var:
                    index = _find_index(origin_var, word)
                    original_dup[i][j][k] = func_var[index]
                if word not in KEYWORD and word.isdigit():
                    original_dup[i][j][k] = 'CONST'
    return original_dup, func_var

## Create Pure list  
'var', 'CONST'와 같은 문자들로 치환되지 않고, 기존의 텍스트에서  
키워드 및 변수, 상수 별로만 나눈 리스트를 만드는 함수  

In [14]:
 def Create_pure_list(line):
    pure_list = []
    for i in range(len(line)):
        pure_list.append(line[i].split())
    for i, list1d in enumerate(pure_list):
        for j, str_ in enumerate(list1d):
            if str_ == 'TAB':
                pure_list[i][j] = '\t'
    return pure_list

Ex)   
``` python
>>> pure_list   
[['for', 'i', 'in', 'range', '(', '10', ')', ':'],
 ['a', '=', 'b', '+', 'c']]
```

## Create Var List
코드에서 사용된 모든 변수에 대한 리스트와  
해당 변수에 매핑할 'var' + n 형식의 리스트를 만드는 함수   

In [15]:
def Create_var_list(pure_list):
    origin_var  =   []
    change_var  =   []
    n           =    1
    for i, list1d in enumerate(pure_list):
        for j, str_ in enumerate(list1d):
            if str_ not in KEYWORD and str_.isdigit() == False and str_ not in origin_var: 
                next_index = j + 1
                if next_index < len(list1d) and pure_list[i][next_index] != '(':
                    origin_var.append(str_)
                    change_var.append('var' + str(n))
                    n += 1
                elif next_index == len(list1d):
                    origin_var.append(str_)
                    change_var.append('var' + str(n))
                    n += 1
    return origin_var, change_var

Ex)   
```python
>>> origin_var
['a', 'b', 'c', 'd', 'e']
>>> change_var
['var1', 'var2', 'var3', 'var4', 'var5']
```

## _Text list Preprocess
text_list를 전처리해주는 함수  
(Func_replace()에서 내부적으로 호출)

In [16]:
def _text_list_preprocess(text_list):
    for i, list1d in enumerate(pure_list):
        for j, str_ in enumerate(list1d):
            if j + 1 < len(list1d) and text_list[i][j + 1] == '(':
                continue
            elif str_ not in KEYWORD and str_.isdigit() == False:
                text_list[i][j] = 'var'
            elif str_ not in KEYWORD and str_.isdigit() == True:
                text_list[i][j] = 'CONST'

## _Create Params  
치환해줄 함수 자리의 파라미터를 계산하여 생성해주는 함수  
(Func_replace()에서 내부적으로 호출)

In [17]:
def _Create_Params(pure_list, index, size, origin_var, change_var):
    params = []
    start = True
    for i in range(index, index + size):
        for var in origin_var:
            if var in pure_list[i] and var not in params:
                if start == True:
                    params.append(var)
                    start = False
                else:
                    params.append(', ')
                    params.append(var)
    return params

Ex)  
```python
>>> _Create_Params(pure_list, index, size, origin_var, change_var)
['a', 'b', 'c', 'd', 'e', 'f']
```

## Func Replace  
함수를 생성한 기존 코드 자리에 적절한 함수 호출로 치환해주는 함수  

In [18]:
def Func_replace(duplicated, text_list, pure_list, origin_var, change_var):
    _text_list_preprocess(text_list)
    while True:
        for i, list2d in enumerate(duplicated):
            isDup = True
            index = 0
            start = True
            for list1d in list2d:
                if list1d not in text_list:
                    isDup = False
                elif list1d in text_list and start == True:
                    index = text_list.index(list1d)
                    start = False
            if isDup == True:
                func_replace = ['func']
                func_replace[0] += str(i + 1)
                func_replace.extend('(')
                func_replace.extend(_Create_Params(pure_list, index,len(list2d), origin_var, change_var))
                func_replace.extend(')')
                pure_list[index] = func_replace
                text_list[index] = func_replace
                for i in range(len(list2d) - 1):
                    del pure_list[index + 1]
                    del text_list[index + 1]
            else :
                return

## Create Code
최종적으로 겹치는 코드를 함수화 및 함수호출로 치환까지 마친 코드를 생성하는 함수

In [19]:
def Create_code(pure_list, func_list):
    code = ""
    for item in func_list:
        code += item
    code += _Insert_Space_to_match_grammar(pure_list,'')
    return code

# MainEntry
위에 정의한 함수들을 호출하여 사용하는 메인 함수

## Input

In [20]:
open_text                   =    open('c:/app/test.txt','r')
text                        =    open_text.read()
preprocessed                =    Preprocess(text)
line                        =    Text_to_line(preprocessed)
pure_list                   =    Create_pure_list(line)
origin_var, change_var      =    Create_var_list(pure_list)
changed, original           =    Line_to_key(line)
duplicated, original_dup    =    Check_duplicate(changed, original)
duplicate, original_dup     =    del_duplicate(duplicated, original_dup)
func_ready, func_var        =    check_same_var(original_dup)
func_list, func_name_list   =    Create_func(func_ready, func_var)
text_list                   = copy.deepcopy(pure_list)
Func_replace(duplicated, text_list, pure_list, origin_var, change_var)
code                        =    Create_code(pure_list, func_list)
print(code)
open_text.close()

def func1(var1, var2, var3, var4) : 
	var1 = str 
	for var2 in var3 : 
		 var1 += var2 
	var1 += _Insert_Space_to_match_grammar(var4 , str) 
def func2(var5, var6, var7, var8, var9, var10) : 
	var5 = var6 + var7 
	var6 = var5 + var7 
	var8 = var9 + var10 
def func3(var11, var12, var13, var14, var15, var16, var17, var18) : 
	for var11 in var12 : 
		 for var13 in var11 : 
		 	 for var14 in var13 : 
		 	 	 if var14 not in var15 and not var14.isdigit() and var14 not in var16 : 
		 	 	 	 var16.append(var14) 
		 	 	 	 var17.append(str + str(var18) ) 
		 	 	 	 var18 += CONST 
func1(code ,  item ,  func_list ,  pure_list) 
func2(a ,  b ,  c ,  d ,  e ,  f) 
for i in range(10) : 
	 print(str) 
func3(func ,  original_dup ,  line ,  word ,  KEYWORD ,  origin_var ,  change_var ,  cnt) 
func2(z ,  x ,  y ,  m ,  o ,  p) 
func3(func ,  original_dup ,  line ,  word ,  KEYWORD ,  origin_var ,  change_var ,  cnt) 
func1(code2 ,  item ,  func_list ,  code ,  pure_list) 



In [27]:
origin_var

['a', 'b', 'c', 'arr', 'i', 'j', 'x', 't', 'y']

In [28]:
change_var

['var1', 'var2', 'var3', 'var4', 'var5', 'var6', 'var7', 'var8', 'var9']

In [24]:
list_ = [['a', '=', 'b', '+', 'c'],
         ['b', '=', 'a', '+', 'c'],
         ['d', '=', 'e', '+', 'f']]
import numpy as np
print(np.array(list_))

[['a' '=' 'b' '+' 'c']
 ['b' '=' 'a' '+' 'c']
 ['d' '=' 'e' '+' 'f']]


In [25]:
print([['func', '1', '(', 'a', 'b', 'c', 'd', 'e', 'f' ,')']])

[['func', '1', '(', 'a', 'b', 'c', 'd', 'e', 'f', ')']]


In [ ]:
a = b + c
b = a + c
d = e + f

for i, list1d in enumerate(pure_list):
    for j, str_ in enumerate(list1d):
        if j + 1 < len(list1d) and text_list[i][j + 1] == '(':
            continue
        elif str_ not in KEYWORD and str_.isdigit() == False:
            text_list[i][j] = 'var'
        elif str_ not in KEYWORD and str_.isdigit() == True:
            text_list[i][j] = 'CONST'

x = y + z
y = x + z
n = o + p

for i, list1d in enumerate(pure_list):
    for j, str_ in enumerate(list1d):
        if j + 1 < len(list1d) and text_list[i][j + 1] == '(':
            continue
        elif str_ not in KEYWORD and str_.isdigit() == False:
            text_list[i][j] = 'var'
        elif str_ not in KEYWORD and str_.isdigit() == True:
            text_list[i][j] = 'CONST'